# GBF approximant with optimized parameters

This demo is a repetition of `demo_interpolation`, but with parameter optimization.

In [ ]:
from utils import plot_graph
from graph_loaders import load_graph
import matplotlib.pyplot as plt
import numpy as np
from approx import GBFInterpolant
from kernels import VarSpline, Diffusion
import networkx as nx
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from itertools import product

### Load a graph

We start by loading a pre-defined graph to be used as an example. 

All the following graphs have coordinate information for each node (as an attribute `pos` scaled to `[0, 1]^2`) that is used for visualization purposes. However, this information is not necessary nor used in the approximation process, since the main code only assumes that `G` is a `networkx` graph.

In [ ]:
# G = load_graph('wbc')
# G = load_graph('sensor2')
# G = load_graph('sensor1')
# G = load_graph('emptyset')
# G = load_graph('2moon')
# G = load_graph('minnesota')
# G = load_graph('rand')
# G = load_graph('rand_sparse')
G = load_graph('bunny')

### Define a training and a test set

We define a signal/function on the nodes using the `pos` attribute. This is an interesting test as the approximation process does not have access to this attribute, and it tries to reconstruct the signal by using only information on the nodes' connectivity.

The signal `f` is defined as a Gaussian centered and scaled around the mean point of the graph.

In [ ]:
f = lambda x: np.exp(-(4 * np.linalg.norm(x - [.5, .5], axis=1)) ** 2)

We extract a random subset of 10% of the nodes to be used as the training set, and as a test set we use the entire set of nodes. All nodes sets are represented by the list of their indices in the graph.

In [ ]:
n_train = int(len(G) * 0.1)
X_train = np.random.randint(1, len(G), size=n_train)
X_test = np.arange(len(G))

Then, we assign the train and test values by evaluating `f`.

In [ ]:
pos = np.array([[pos[0], pos[1]] for pos in nx.get_node_attributes(G, 'pos').values()])

y_test = np.array(f(pos))
y_train = y_test[X_train]

### Optimize the parameters and reconstruct the signal

We first define a metric to rank the performances of the different parameters. In this case the best model is the one providing the smallest mean error.

In [ ]:
def mean_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

scorer = make_scorer(mean_error, greater_is_better=False)

We then specify the parameter grid to be evaluated. To optimize both the kernel parameter and the regularization parameter with a fixed kernel, it is sufficient to specify the range of each single parameter.

In [ ]:
# params = {
#         'reg_par': np.logspace(-15, 0, 5),
#         'kernel_par': [[-x] for x in np.logspace(-1, 2, 5)]
# }

For kernels with multiple `kernel_par` it is possible to use instead something like the following.

In [ ]:
# params = {
#         'reg_par': np.logspace(-15, 0, 5),
#         'kernel_par': [[-x, y] for x in np.logspace(-1, 2, 5) for y in np.linspace(0, 10, 5)]
#         }

In this case we treat also the kernel as a parameter to be optimized. Since different kernels may have different parameter number and range, we explicitly build the `params` serch grid by discretizing each parameter with `n_grid` samples.

In [ ]:
n_grid = 5

# Grid for the Diffusion kernel
kernel = [['Diffusion']]
reg_par = [[x] for x in np.logspace(-15, 0, n_grid)]
kernel_par = [[-x] for x in np.logspace(-1, 2, n_grid)]
params_1 = [{'kernel': kernel, 'reg_par' : reg_par, 'kernel_par': kernel_par}
                   for kernel, reg_par, kernel_par in product(kernel, reg_par, kernel_par)]

# Grid for the VarSpline kernel
kernel = [['VarSpline']]
reg_par = [[x] for x in np.logspace(-15, 0, n_grid)]
kernel_par = [[-x, y] for x in np.logspace(-1, 2, 5) for y in np.linspace(0, 10, n_grid)]
params_2 = [{'kernel': kernel, 'reg_par' : reg_par, 'kernel_par': kernel_par}
                   for kernel, reg_par, kernel_par in product(kernel, reg_par, kernel_par)]

# Join the two grids
params = params_1 + params_2

In this case, we wrap the approximation model into a `GridSearchCV`. We use all the available cores and run `cv=5`-fold cross validation, with final refitting.

In [ ]:
model = GridSearchCV(GBFInterpolant(G, verbose=False), 
                     params, scoring=scorer, n_jobs=-1, cv=5, 
                     refit=True, verbose=1)

In [ ]:
model.fit(X_train, y_train)

Finally, we visualize the selected parameters.

In [ ]:
import pandas as pd
pd.DataFrame(model.cv_results_).sort_values(by='rank_test_score').head(5)

In [ ]:
model.best_params_

### Compute the model predictions

Now that the model is trained, we can compute the predictions on the test set exactly as before.

In [ ]:
s_test = model.predict(X_test)

And compute some errors. We use a clipping in the computation of the relative error to avoid dividing by zero.

In [ ]:
rel_err_tol = 1e-10
abs_err_test = np.abs(y_test - s_test)
rel_err_test = abs_err_test / np.clip(np.abs(y_test), rel_err_tol, np.inf)

### Visualize

Finally, we visualize some results: the original and the reconstructed signal.

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = plt.subplot(1, 2, 1)
plot_graph(G, ax=ax, values=y_test, nodelist=model.best_estimator_.ctrs_, 
           cb_label='Target signal')

ax = plt.subplot(1, 2, 2)
plot_graph(G, ax=ax, values=s_test, nodelist=model.best_estimator_.ctrs_, 
           cb_label='Reconstructed signal')

And the absolute and relative test errors.

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = plt.subplot(1, 2, 1)
plot_graph(G, ax=ax, values=abs_err_test, nodelist=model.best_estimator_.ctrs_, 
           cb_label='Absolute Error', log_scale=True)

ax = plt.subplot(1, 2, 2)
plot_graph(G, ax=ax, values=rel_err_test, nodelist=model.best_estimator_.ctrs_, 
           cb_label='Relative Error (clipped to %2.2e)' % rel_err_tol, log_scale=True)